# Notebook 4 : Création d'un dataset d'évaluation

L'objectif de ce notebook est de créer une un dataset qui contiendra des questions potentielles que l'utilisateur pourrait poser à notre système.

En résultat il contiendra les réponse attendues.

Etant donné que nous avons un système de recherche vectorielle, il se pourrait qu'une question ait plusieurs bonnes réponses possibles.

C'est pourquoi dans notre dataset d'évaluation, nous aurons une colonne `question` et une colonnes `relevent_cocktails` qui contiendront les 10 cocktails les plus pertinents pour la question donnée. De plus nous aurons une colonne `category` pour identifier le type de question

## Etape 1 : Identification des cas d'usage

Premièrement, nous allons identifier tous les cas d'usage de notre système pour pouvoir générer des questions :

- **Recherche par nom de cocktail** : "Quelle est la recette du cocktail Margarita ?"
- **Recherche par ingrédient** : "Quels cocktails puis-je faire avec du rhum et du citron ?"
- **Recherche par catégorie** : "Quel cocktail je pourrais faire sans alcool ?" 
- **Recherche par type de verre** : "Quels cocktails puis-je faire avec un verre à Martini ?"
- **Recherche par caractéristique** : "Je cherche un cocktail rafraichissant pour l'été"
- **Recherche par occasion** : "Quels cocktails puis-je faire pour une soirée entre amis ?"

## Etape 2 : Génération des questions

Maintenant que nous avons nos cas d'usage, nous allons générer 15 questions grâce à un LLM, ce qui va nous permettre de générer des questions de manière automatique et de manière variée.

C'est aussi ce LLM, qui va nous fournir les 10 cocktails les plus pertinents pour chaque question.

Le LLM que nous allons utiliser est Llama3.2 instancié avec ollama.

In [ ]:
import pandas as pd
import os
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_csv('../data/cocktails.csv') 

api_key = os.getenv('MISTRAL_API_KEY')
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

def request_mistral(prompt):
    chat_response = client.chat.complete(
        model=model,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ],
        max_tokens=1000,
        temperature=0.7
    )
    return chat_response.choices[0].message.content

categories = {
    "Recherche par nom de cocktail": {
        "description": "Find cocktails by exact name match",
        "examples": ["Quelle est la recette du cocktail Margarita ?", "Comment faire un Mojito ?"]
    },
    "Recherche par ingrédient": {
        "description": "Find cocktails containing specific ingredients",
        "examples": ["Quels cocktails puis-je faire avec du rhum et du citron ?", "J'ai de la vodka et du jus d'orange, quel cocktail puis-je faire ?"]
    },
    "Recherche par catégorie": {
        "description": "Find cocktails by category (e.g., non-alcoholic)",
        "examples": ["Quel cocktail je pourrais faire sans alcool ?", "Quels cocktails sont à base de champagne ?"]
    },
    "Recherche par type de verre": {
        "description": "Find cocktails served in specific glassware",
        "examples": ["Quels cocktails puis-je faire avec un verre à Martini ?", "J'ai des verres Highball, quel cocktail faire ?"]
    },
    "Recherche par caractéristique": {
        "description": "Find cocktails by specific characteristics",
        "examples": ["Je cherche un cocktail rafraichissant pour l'été", "Quel cocktail fort en alcool ?"]
    },
    "Recherche par occasion": {
        "description": "Find cocktails for specific occasions",
        "examples": ["Quels cocktails puis-je faire pour une soirée entre amis ?", "Je cherche un cocktail pour un anniversaire"]
    }
}

def generate_questions(category, examples):
    prompt = f"""
    Generate 15 diverse questions using different wording/formulation in French about cocktails for this category: {category}
    Examples: {examples}
    
    Return only the questions in this format, and do not include a introductory sentence in your response:
    1. [question]
    2. [question]
    ...
    15. [question]
    """
    
    response = request_mistral(prompt)

    return [line.split('. ', 1)[1] for line in response.split('\n') if line.strip()]

def find_relevant_cocktails(question):
    prompt = f"""
    Select the 10 most relevant cocktails from this list for the question: 
    Question: {question}
    
    Cocktail names:
    {', '.join(df['name'].tolist())}
    
    Return only the names separated by commas, don't include a introductory sentence in your answer.
    """
    
    response = request_mistral(prompt)
    
    return [name.strip() for name in response.split(', ')]

## Etape 3 : Génération du dataset

Désormais que nous avons nos méthodes de génération de questions et leurs réponses associées, nous allons générer notre dataset d'évaluation.

In [39]:
import time

dataset = pd.DataFrame(columns=["category", "question", "relevant_cocktails"])

for category, details in categories.items():
    print(f"Génération des questions de la categorie: {category}")
    
    questions = generate_questions(category, details["examples"])
    
    time.sleep(1)

    for question in questions:        
        best_cocktails = find_relevant_cocktails(question)

        dataset = pd.concat([dataset, pd.DataFrame({
            "category": [category],
            "question": [question],
            "relevant_cocktails": ['; '.join(best_cocktails)]
        })], ignore_index=True)
        time.sleep(1)

Génération des questions de la categorie: Recherche par nom de cocktail
Génération des questions de la categorie: Recherche par ingrédient
Génération des questions de la categorie: Recherche par catégorie
Génération des questions de la categorie: Recherche par type de verre
Génération des questions de la categorie: Recherche par caractéristique
Génération des questions de la categorie: Recherche par occasion


## Etape 4 : Sauvegarde du dataset

Désormais que notre dataset est généré, nous allons le sauvegarder en format csv pour pouvoir l'utiliser dans le notebook suivant.

In [ ]:
dataset.to_csv('../data/cocktails_questions_eval.csv', index=False)